In [1]:
import requests
import pandas as pd
import re
import time
import os
from lxml import html
import urllib.parse
import glob
import openai
from tqdm.notebook import tqdm
from pydantic import BaseModel, field_validator
from datetime import datetime
from openai import OpenAI

# You need to turn on the VPN and use the OpenAI API key


In [ ]:
# Create an OpenAI client instance (ensure your credentials are set up).
from typing import List
client = OpenAI(api_key="your openai apik key here") 


class Stamp(BaseModel):
    """Represents a single stamp/bounding box in an image."""
    x: float  # X-coordinate of the top-left corner
    y: float  # Y-coordinate of the top-left corner
    width: float  # Width of the stamp
    height: float  # Height of the stamp

class StampDetectionResult(BaseModel):
    """Container for multiple stamps detected in an image."""
    stamps: List[Stamp] = []  # List of detected stamps

    def add_stamp(self, x: float, y: float, width: float, height: float) -> None:
        """Helper method to add a stamp to the list."""
        self.stamps.append(Stamp(x=x, y=y, width=width, height=height))

class StampInfo(BaseModel):
    title: str
    issue_date: str  # should be in 'yyyy-mm-dd' format
    width_size: float
    height_size: float
    factory: str  # e.g. "西川邮政管理局"
    designer: str  # Designer of the stamp
    printing_method: str  # Printing method used for the stamp

    @field_validator("issue_date")
    def validate_issue_date(cls, v):
        try:
            datetime.strptime(v, "%Y-%m-%d")
        except ValueError:
            raise ValueError("issue_date must be in 'yyyy-mm-dd' format")
        return v


def extract_stamp_info(description):
    """
    Uses OpenAI's Structured Outputs to extract stamp metadata from the description.
    The model is instructed to extract the following keys:
      - title
      - issue_date
      - width_size
      - height_size
      - designer
      - printing_method
      - factory
      
    """
    messages = [
        {
            "role": "system",
            "content": (
                "Extract stamp metadata from the description and return JSON with these keys: "
                "title, issue_date, width_size, height_size, factory, designer, and printing_method. "
                "Sizes are in mm. If issue_date is just a year, append '-01-01' to complete the date."
            ),
        },
        {"role": "user", "content": description},
    ]

    # Use the structured outputs feature via response_format
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=messages,
        # response_format=StampInfo,
        response_format=StampDetectionResult
    )
    # The parsed response is available from the first choice
    return completion.choices[0].message.parsed